In [6]:
#### Basic Setting

import  load_dotenv
load_dotenv.load_dotenv("../../All_LLM_tutorial/.env")

True

In [10]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
)
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import chromadb
from llama_index.core.node_parser import SentenceSplitter


Settings.llm = OpenAI(model="gpt-4o-mini")
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

In [11]:
chroma_client = chromadb.PersistentClient(path="./chroma_db_example")
chroma_collection = chroma_client.get_or_create_collection("cloud")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)

In [14]:
from llama_index.core.agent.workflow import ReActAgent
from llama_index.core.workflow import Context


from llama_index.core.tools import QueryEngineTool

query_engine_tools = [
    QueryEngineTool.from_defaults(
        query_engine=index.as_query_engine(),
        name="cloud",
        description=(
            "KT cloud 사용법 설명"
        ),
    ),

]



In [15]:
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core.llms import ChatMessage
from llama_index.core.memory import Memory


memory = Memory.from_defaults(session_id="my_session", token_limit=40000)
memory.put_messages(
    [
        ChatMessage(role="user", content="Hello, world!"),
        ChatMessage(role="assistant", content="Hello, world to you too!"),
    ]
)
chat_history = memory.get()

agent = FunctionAgent(tools=query_engine_tools)

# passing in the chat history overrides any existing memory
response = await agent.run(
    "<question that invokes tool>", chat_history=chat_history
)

In [19]:
str(response)

'It seems like you might want to ask a specific question or invoke a tool. Please provide more details or specify what you need help with!'

In [20]:
await agent.run(
    "SSH 접속 방법", chat_history=chat_history
)

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='SSH 접속 방법은 다음과 같습니다:\n\n1. **SSH/SFTP 앱 실행**: 세션의 컨트롤 열에 있는 App 아이콘을 클릭하여 앱을 실행합니다.\n2. **세션 정보 확인**: SSH/SFTP 앱을 선택하면 세션을 로컬에서 접근할 수 있는 주소와 예시가 표시됩니다. 이 정보를 활용하여 로컬 파일을 전송할 수 있습니다.\n3. **SSH 클라이언트 사용**: NOTEPutty와 같은 SSH 클라이언트를 사용하여 외부에서 SSH 프로토콜로 세션에 접속할 수 있습니다.\n\n더 자세한 내용은 제공된 링크를 참고하세요. 추가적인 질문이 있으시면 언제든지 말씀해 주세요!')]), tool_calls=[ToolCallResult(tool_name='cloud', tool_kwargs={'input': 'SSH 접속 방법에 대해 설명해 주세요.'}, tool_id='call_Up0PUnizSvRQ5gLjx3OEtdtU', tool_output=ToolOutput(blocks=[TextBlock(block_type='text', text='SSH 접속 방법은 다음과 같습니다. 세션의 컨트롤 열에 있는 App 아이콘을 클릭하여 앱을 실행한 후, SSH/SFTP 앱을 선택합니다. 그러면 세션을 로컬에서 접근할 수 있는 주소와 예시가 표시됩니다. 이 정보를 활용하여 로컬 파일을 전송할 수 있습니다. 또한, NOTEPutty와 같은 툴을 사용하여 외부에서 SSH 프로토콜로 세션에 접속하는 방법에 대한 자세한 내용은 제공된 링크를 참고하면 됩니다.')], tool_name='cloud', raw_input={'input': 'SSH 접속 방법에 대해 설명해 주세요.'}, raw_output=Response(response='SSH

In [22]:
from llama_index.core.workflow import Context

ctx = Context(agent)

response = await agent.run("SSH 접속방법은?", ctx=ctx)

# get the memory
memory = await ctx.store.get("memory")
chat_history = memory.get()

In [35]:
memory = Memory.from_defaults(
    session_id="my_session",
    token_limit=40000,
    chat_history_token_ratio=0.7,
    token_flush_size=3000,
    # async_database_uri="sqlite:///memory.db"
    # 장기 메모리 관리하려면 sqlalchemy
)

In [36]:
response = await agent.run("Hello!", memory=memory)